## Import the libraries, load dataset, print shape of data, data description.

In [81]:
import pandas as pd
from bs4 import BeautifulSoup
import contractions
import re, string, unicodedata                          # Import Regex, string and unicodedata.
from nltk.stem.wordnet import WordNetLemmatizer
df = pd.read_csv('Tweets.csv')
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics import accuracy_score
from nltk.tokenize import sent_tokenize, word_tokenize

In [82]:
pd.set_option('display.max_colwidth', None)
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


## Understand of data-columns and drop all colunns but airline_sentiment and text

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [84]:
df.drop(columns=['tweet_id', 'airline_sentiment_confidence','negativereason','negativereason_confidence', 'airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'], inplace=True)

In [85]:
df.shape

(14640, 2)

In [86]:
df.head()


,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials to the experience... tacky.
2,neutral,@VirginAmerica I didn't today... Must mean I need to take another trip!
3,negative,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse"
4,negative,@VirginAmerica and it's a really big bad thing about it


In [87]:
sw = stopwords.words('english')

customlist = ['not', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn',
        "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',
        "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn',
        "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# Set custom stop-word's list as not, couldn't etc. words matter in Sentiment, so not removing them from original data.

stopswords = list(set(sw) - set(customlist))

lemmatizer = WordNetLemmatizer()

def lemmatize_list(words):
    new_words = []
    for word in words:
        new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []

    for word in words:
        if word not in stopswords:
            new_words.append(word)
    return new_words

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


In [88]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)


In [89]:
def remove_handles(text):
    text = re.sub("@\w{1,15}", " ", text)
    return text


In [90]:
def remove_numbers(text):
  text = re.sub(r'\d+', '', text)
  return text

In [91]:
def remove_links(text):
    httpRegex = "(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
    text = re.sub(httpRegex, "", text)
    return text


## Text pre-processing: Data preparation.

In [92]:
# remove twitter handles
df['text'] = df.apply(lambda row: remove_handles(row['text']), axis=1)

# remove http links
df['text'] = df.apply(lambda row: remove_links(row['text']), axis=1)

# remove html tags
df['text'] = df.apply(lambda row: strip_html(row['text']), axis=1)

# remove extra white spaces
df['text'] = df.apply(lambda row: row['text'].strip(), axis=1)

# remove numbers
df['text'] = df.apply(lambda row: remove_numbers(row['text']), axis=1)

# remove contractions
df['text'] = df.apply(lambda row: replace_contractions(row['text']), axis=1)
# df.head()

In [93]:
def normalize(text):
    # tokenize
    words = word_tokenize(text)
    # remove non ascii
    words = remove_non_ascii(words)
    # remove punctuation
    words = remove_punctuation(words)
    # convert to lowercase
    words = to_lowercase(words)
    # remove stop words
    words = remove_stopwords(words)
    # lemmatize
    words = lemmatize_list(words)
    # join back into a sentence
    return ' '.join(words)


In [94]:
df['text'] = df.apply(lambda row: normalize(row['text']), axis=1)

In [95]:
df.head()

,airline_sentiment,text
0,neutral,say
1,positive,plus add commercials experience tacky
2,neutral,not today must mean need take another trip
3,negative,really aggressive blast obnoxious entertainment guests face little recourse
4,negative,really big bad thing


## Vectorization

### Count Vectorizer

In [96]:
from sklearn.feature_extraction.text import CountVectorizer

countVectorizer = CountVectorizer(max_features=5000)                # Keep only 1000 features as number of features will increase the processing time.
data_features = countVectorizer.fit_transform(df['text'])

data_features = data_features.toarray()

In [97]:
data_features.shape

(14640, 5000)

In [98]:
labels = df['airline_sentiment']


### Evaluate Model

In [99]:
# Split data into training and testing set.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_features, labels, test_size=0.3, random_state=42)

In [100]:
# Using Random Forest to build model for the classification of airline sentiment.
# Also calculating the cross validation score.

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest = RandomForestClassifier(n_estimators=10, n_jobs=4)

forest = forest.fit(X_train, y_train)

print(forest)

print(np.mean(cross_val_score(forest, data_features, labels, cv=10)))

RandomForestClassifier(n_estimators=10, n_jobs=4)
0.7407786885245902


In [101]:
y_pred_test = forest.predict(X_test)


### Accuracy Score - Count Vectorizer

In [102]:

accuracy_score(y_test, y_pred_test)

0.7684426229508197

## TFID Vectorizer

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidVectorizer = TfidfVectorizer(max_features=5000)
data_features_tfid = tfidVectorizer.fit_transform(df['text'])

data_features_tfid = data_features_tfid.toarray()

data_features_tfid.shape


(14640, 5000)

### Evaluate Model

In [104]:
X_train_tfid, X_test_tfid, y_train_tfid, y_test_tfid = train_test_split(data_features_tfid, labels, test_size=0.3, random_state=42)

In [105]:
# Using Random Forest to build model for the classification of airline sentiment.
# Also calculating the cross validation score.

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest_tfid = RandomForestClassifier(n_estimators=10, n_jobs=4)

forest_tfid = forest_tfid.fit(X_train_tfid, y_train_tfid)


In [106]:
y_pred_test_tfid = forest_tfid.predict(X_test_tfid)

### Accuracy Score - TFID Vectorizer

In [107]:
accuracy_score(y_test_tfid, y_pred_test_tfid)

0.7670765027322405

## Summary

* We used dataset and narrowed it down to columns for airline_sentiment and text
* The goal was to build a model for text-classification.
* We Pre-processed the data using various techniques and libraries. Including removing twitter handles, links, html tags, nono ascii, numbers, punctuation, stop words, lemmatization
* The pre-precessed data is converted to numbers, so that we can feed the data in the model, using both count and tfid vectorizers.
* After building the classification model, we predicted the result for the test data (for both count and tfid vectorizers).
* Models performed very similarly regardless of how data_features were generated (count and tfid vectorizers) and there was no benefit in using one over the other
* One way to increase accuracy is to try different classifiers and optimize hyper parameters
* One more way to increase accuracy is to use different variations of Pre-processing techniques.